In [ ]:
pip install scipy

In [ ]:
# Importing python libraries
#
import pandas as pd
import numpy as np
from scipy.stats import mode
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set(style = 'darkgrid')
import requests
from io import StringIO
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import export_graphviz 
from IPython.display import Image  
import pydotplus
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
student_data = pd.read_csv('../data/student-mat.csv', sep=";")


In [ ]:
student_data.head()

In [ ]:
# label encode final_grade
le = preprocessing.LabelEncoder()
student_data.final_grade = le.fit_transform(data.final_grade)

In [ ]:
# Drop the target columns from the predictive features
X = student_data.drop(labels=['final_grade','final_score'],axis=1)

In [ ]:
# set the target variable
y = student_data.final_grade

In [ ]:
# get dummy varibles for the predictive features
X = pd.get_dummies(X)


In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state = 101)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

### Logistic Regression

In [ ]:
# define and configure the model
# l1 regularization gives better results
model = LogisticRegression()
# fit the model
model.fit(X_train, y_train)

# evaluate the model
preds = model.predict(X_test)
print('Accuracy : ',accuracy_score(y_test, preds))

print(f'The accuracy score is: {accuracy_score(y_test, preds)} and the f1 score is {f1_score(y_test, preds)}')
print(f'The recall score is: {recall_score(y_test, preds)}')
print('\n')
print(f'{classification_report(y_test, preds)}')
matrix = confusion_matrix(y_test, preds)
print(matrix)

In [ ]:
sns.heatmap(matrix,annot = True, fmt = "d")

### Decision Tree Classifier

In [ ]:

# define and configure the model
model = DecisionTreeClassifier()
# fit the model
model.fit(X_train, y_train)
# evaluate the model
preds = model.predict(X_test)
print('Accuracy : ',accuracy_score(y_test, preds))

print(f'The accuracy score is: {accuracy_score(y_test, preds)} and the f1 score is {f1_score(y_test, preds)}')
print(f'The recall score is: {recall_score(y_test, preds)}')
print('\n')
print(f'{classification_report(y_test, preds)}')
matrix = confusion_matrix(y_test, preds)
print(matrix)

In [ ]:
sns.heatmap(matrix,annot = True, fmt = "d")

### Random Forest Classifier

In [ ]:
# define and configure the model
model = RandomForestClassifier()

# fit the model
model.fit(X_train, y_train)

# evaluate the model
preds = model.predict(X_test)
print('Accuracy : ',accuracy_score(y_test, preds))

print(f'The accuracy score is: {accuracy_score(y_test, preds)} and the f1 score is {f1_score(y_test, preds)}')
print(f'The recall score is: {recall_score(y_test, preds)}')
print('\n')
print(f'{classification_report(y_test, preds)}')
matrix = confusion_matrix(y_test, preds)
print(matrix)

In [ ]:
sns.heatmap(matrix,annot = True, fmt = "d")